In [5]:
pip install pandas openpyxl

Note: you may need to restart the kernel to use updated packages.


## 1. Instalar as bibliotecas necessárias

In [12]:
import pandas as pd
from datetime import time as dt_time
from datetime import datetime

## 2. Importar os arquivos no notebook

In [13]:
# Carregar arquivos excel

flights = pd.read_csv(r"C:\Users\Notebook\Documents\Laboratoria\flights_202301.csv")
airline_dict = pd.read_csv(r"C:\Users\Notebook\Documents\Laboratoria\AIRLINE_CODE_DICTIONARY.csv")
dot_dict = pd.read_excel(r"C:\Users\Notebook\Documents\Laboratoria\DOT_CODE_DICTIONARY.xlsx")

# conferir 

print("Voos:")
display(flights.head())

print("Dicionário de companhias aéreas:")
display(airline_dict.head())

print("Dicionário DOT:")
display(dot_dict.head())

Voos:


,FL_DATE,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,CRS_DEP_TIME,DEP_TIME,...,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT,FL_YEAR,FL_MONTH,FL_DAY
0,2023-01-02,9E,20363,4628,BDL,"Hartford, CT",LGA,"New York, NY",800,757.0,...,25.0,101,NaN,NaN,NaN,NaN,NaN,2023,1,2
1,2023-01-03,9E,20363,4628,BDL,"Hartford, CT",LGA,"New York, NY",800,755.0,...,37.0,101,NaN,NaN,NaN,NaN,NaN,2023,1,3
2,2023-01-04,9E,20363,4628,BDL,"Hartford, CT",LGA,"New York, NY",800,755.0,...,28.0,101,NaN,NaN,NaN,NaN,NaN,2023,1,4
3,2023-01-05,9E,20363,4628,BDL,"Hartford, CT",LGA,"New York, NY",800,754.0,...,38.0,101,NaN,NaN,NaN,NaN,NaN,2023,1,5
4,2023-01-06,9E,20363,4628,BDL,"Hartford, CT",LGA,"New York, NY",800,759.0,...,28.0,101,NaN,NaN,NaN,NaN,NaN,2023,1,6


Dicionário de companhias aéreas:


,Code,Description
0,02Q,Titan Airways
1,04Q,Tradewind Aviation
2,05Q,"Comlux Aviation, AG"
3,06Q,Master Top Linhas Aereas Ltd.
4,07Q,Flair Airlines Ltd.


Dicionário DOT:


,Code,Description
0,19031,Mackey International Inc.: MAC
1,19032,Munz Northern Airlines Inc.: XY
2,19033,Cochise Airlines Inc.: COC
3,19034,Golden Gate Airlines Inc.: GSA
4,19035,Aeromech Inc.: RZZ


## 3. Conferir informações iniciais

In [89]:
# Ajustar nomes das colunas
airline_dict = airline_dict.rename(
    columns=lambda col: f"AIRLINE_{col}" 
)

dot_dict = dot_dict.rename(
    columns=lambda col: f"DOT_{col}" 
)

In [104]:
airline_dict = airline_dict.rename(
    columns={"AIRLINE_AIRLINE_AIRLINE_{col}": "AIRLINE_Description"}
)

dot_dict = dot_dict.rename(
    columns={"DOT_{col}": "DOT_Description"}
)

In [56]:
# Ver dimensões
print("Voos:", flights.shape)
print("Airline dict:", airline_dict.shape)
print("DOT dict:", dot_dict.shape)

Voos: (538837, 33)
Airline dict: (1729, 2)
DOT dict: (1737, 2)


In [81]:
pd.set_option("display.max_rows", None)

# Resumo das colunas
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 538837 entries, 0 to 538836
Data columns (total 33 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   FL_DATE                  538837 non-null  object 
 1   AIRLINE_CODE             538837 non-null  object 
 2   DOT_CODE                 538837 non-null  int64  
 3   FL_NUMBER                538837 non-null  int64  
 4   ORIGIN                   538837 non-null  object 
 5   ORIGIN_CITY              538837 non-null  object 
 6   DEST                     538837 non-null  object 
 7   DEST_CITY                538837 non-null  object 
 8   CRS_DEP_TIME             538837 non-null  int64  
 9   DEP_TIME                 528859 non-null  float64
 10  DEP_DELAY                528855 non-null  float64
 11  TAXI_OUT                 528640 non-null  float64
 12  WHEELS_OFF               528640 non-null  float64
 13  WHEELS_ON                528318 non-null  float64
 14  TAXI

In [92]:
airline_dict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1729 entries, 0 to 1728
Data columns (total 2 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   AIRLINE_Code                 1728 non-null   object
 1   AIRLINE_AIRLINE_Description  1729 non-null   object
dtypes: object(2)
memory usage: 27.1+ KB


In [99]:
dot_dict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1737 entries, 0 to 1736
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   DOT_Code         1737 non-null   int64 
 1   DOT_Description  1733 non-null   object
dtypes: int64(1), object(1)
memory usage: 27.3+ KB


## 4. Ajustar tipo de dados

4.1. Flights

In [ ]:
def optimize_flight_data_types(df):
    df_optimized = df.copy()
    # Cria uma cópia do dataframe original para não alterar df em memória
    ## Preserva o original caso seja necessário reverter ou comparar

    # 1. Converter datas
    if "FL_DATE" in df_optimized.columns:
        df_optimized["FL_DATE"] = pd.to_datetime(
            df_optimized["FL_DATE"], format="%Y-%m-%d", errors="coerce"
        )
        ## procura a coluna, se existir, converte para data. 
        ### mostra o formato esperado (ex.: 2023-01-05)
        #### errors='coerce' transforma números inválidos em nulos

    # 2. Converter para categóricas
    cols_category = ["AIRLINE_CODE", "ORIGIN", "ORIGIN_CITY",
                     "DEST", "DEST_CITY", "CANCELLATION_CODE"]
    for col in cols_category:
        if col in df_optimized.columns:
            df_optimized[col] = df_optimized[col].astype("category")
        ## para cada coluna da lista, se ela existir, converte para category, pois reduz o uso da memória e acelera operações
        ### os valores permanecem como texto, mas internamente são armazenados como códigos inteiros + mapeamento 
    
    # 3. Converter variáveis booleanas
    if "CANCELLED" in df_optimized.columns:
        df_optimized["CANCELLED"] = df_optimized["CANCELLED"].astype("boolean")
    if "DIVERTED" in df_optimized.columns:
        df_optimized["DIVERTED"] = df_optimized["DIVERTED"].astype("boolean")
        ## os NA são presercados ao utilizadas .astype("boolean")

    # 4. Ajustar a conversão de tempo e todas as colunas de tempo
    def hhmm_to_time_optimized(val):
        """Convert HHMM format to time, with robust error handling"""
        if pd.isna(val):
            return pd.NaT
        try:
            val = int(val)
            if val == 2400:
                val = 0
            elif val > 2400:
                return pd.NaT
            hh = val // 100
            mm = val % 100
            if hh > 23 or mm > 59:
                return pd.NaT
            return time(hh, mm)
        except (ValueError, TypeError):
            return pd.NaT
         ## converte valors no formato HHMM para um objeto {time(hh,mm)}
    time_columns = [
        "CRS_DEP_TIME", "DEP_TIME", "WHEELS_OFF", "WHEELS_ON",
        "CRS_ARR_TIME", "ARR_TIME"
    ]
    for col in time_columns:
        if col in df_optimized.columns:
            df_optimized[col] = df_optimized[col].apply(hhmm_to_time_optimized)
        ## para cada coluna do tempo presente, aplica a função, para que essas colunas possam conter datetime.time ou pd.NaT
                    
    # 5. Converter colunas numéricas inteiras
    int_columns_small = ["FL_NUMBER", "DOT_CODE", "FL_YEAR", "FL_MONTH", "FL_DAY"]
    for col in int_columns_small:
        if col in df_optimized.columns:
            max_val = df_optimized[col].max()
            if max_val <= 32767:  # int16
                df_optimized[col] = df_optimized[col].astype("int16")
            elif max_val <= 2147483647:  # int32
                df_optimized[col] = df_optimized[col].astype("int32")
    
    if "DISTANCE" in df_optimized.columns:
        df_optimized["DISTANCE"] = df_optimized["DISTANCE"].astype("int16")
        ## para cada coluna inteira listada, pega o max e decide qual tipo de inteiro menor cabe (16 ou 32)
        ### o objetivo é reduzi memória usando inteiros menores quando possível

    # 6. Converter colunas numéricas com casas decimais
    float_columns = [
        "DEP_DELAY", "TAXI_OUT", "TAXI_IN", "ARR_DELAY",
        "CRS_ELAPSED_TIME", "ELAPSED_TIME", "AIR_TIME",
        "DELAY_DUE_CARRIER", "DELAY_DUE_WEATHER", "DELAY_DUE_NAS",
        "DELAY_DUE_SECURITY", "DELAY_DUE_LATE_AIRCRAFT"
    ]
    for col in float_columns:
        if col in df_optimized.columns:
            df_optimized[col] = pd.to_numeric(df_optimized[col], errors="coerce").astype("float32")
        ## para cada coluna, converte primeiro em numérico e depois apra float32
        ### reduz o uso de memória mantendo precisão adequada para análise exploratória e modelagem básica

    return df_optimized


In [44]:
# Aplicar no dataframe flights
flights_optimized = optimize_flight_data_types(flights)

In [45]:
# Verificar resultado
flights_optimized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 538837 entries, 0 to 538836
Data columns (total 33 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   FL_DATE                  538837 non-null  datetime64[ns]
 1   AIRLINE_CODE             538837 non-null  category      
 2   DOT_CODE                 538837 non-null  int16         
 3   FL_NUMBER                538837 non-null  int16         
 4   ORIGIN                   538837 non-null  category      
 5   ORIGIN_CITY              538837 non-null  category      
 6   DEST                     538837 non-null  category      
 7   DEST_CITY                538837 non-null  category      
 8   CRS_DEP_TIME             538837 non-null  object        
 9   DEP_TIME                 528859 non-null  object        
 10  DEP_DELAY                528855 non-null  float32       
 11  TAXI_OUT                 528640 non-null  float32       
 12  WHEELS_OFF      

4.2. airline_dict

In [68]:
df_airline_optimized = airline_dict.copy()

df_airline_optimized['Code'] = df_airline_optimized['Code'].astype('category')

# 2. Convert 'AIRLINE_Description' to category
df_airline_optimized['AIRLINE_Description'] = df_airline_optimized['AIRLINE_Description'].astype('category')

In [69]:
df_airline_optimized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1729 entries, 0 to 1728
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   Code                 1728 non-null   category
 1   AIRLINE_Description  1729 non-null   category
dtypes: category(2)
memory usage: 163.0 KB


4.3. dot_dict

In [70]:
df_dot_optimized = dot_dict.copy()

df_dot_optimized['Code'] = pd.to_numeric(df_dot_optimized['Code'], errors='coerce')

max_dot_code = df_dot_optimized['Code'].max()

if max_dot_code <= 32767:  
    df_dot_optimized['Code'] = df_dot_optimized['Code'].astype('int16')
elif max_dot_code <= 2147483647: 
    df_dot_optimized['Code'] = df_dot_optimized['Code'].astype('int32')

df_dot_optimized['DOT_Description'] = df_dot_optimized['DOT_Description'].astype('category')

In [71]:
df_dot_optimized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1737 entries, 0 to 1736
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   Code             1737 non-null   int16   
 1   DOT_Description  1733 non-null   category
dtypes: category(1), int16(1)
memory usage: 85.0 KB


## 5. Unir Tabelas

Análise das variáveis AIRLINE_CODE e DOT_CODE na tabela flights

In [85]:
flights_optimized["AIRLINE_CODE"].unique()

['9E', 'AA', 'AS', 'B6', 'DL', ..., 'WN', 'YX', 'OH', 'OO', 'UA']
Length: 15
Categories (15, object): ['9E', 'AA', 'AS', 'B6', ..., 'OO', 'UA', 'WN', 'YX']

In [86]:
flights_optimized["DOT_CODE"].unique()

array([20363, 19805, 19930, 20409, 19790, 20436, 20368, 19690, 20398,
       20416, 19393, 20452, 20397, 20304, 19977], dtype=int16)

In [107]:
# Merge com a tabela de voos
df_merged = pd.merge(
    flights_optimized,
    df_dot_optimized,
    on="DOT_CODE",
    how="left"
)

df_merged = pd.merge(
    df_merged,
    df_airline_optimized,
    on="AIRLINE_CODE",
    how="left"
)

KeyError: 'DOT_CODE'